<a href="https://colab.research.google.com/github/Beytab/RS6/blob/master/gbdt%2Blr_safe_driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#安装kaggle

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"yanzaihang","key":"c7d6a781f8fdeed15a844ad2524e1f38"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# 设置数据集文件夹

In [4]:
!mkdir -p safe_driver_data
!cd safe_driver_data
!pwd

/content


# 下载数据

In [5]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 71.3MB/s]
 81% 36.0M/44.4M [00:00<00:00, 48.7MB/s]
100% 44.4M/44.4M [00:00<00:00, 100MB/s] 
 93% 28.0M/30.0M [00:00<00:00, 46.7MB/s]
100% 30.0M/30.0M [00:00<00:00, 75.8MB/s]


In [6]:
!kaggle -v

Kaggle API 1.5.4


In [27]:
!pip install --upgrade --force-reinstall --no-deps kaggle #API版本修正

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=79fcb440e800606478627d063f7139ed23860624a0f72bf1601ce1fa9306e08f
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [28]:
!kaggle -v

Kaggle API 1.5.6


In [29]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

 85% 65.0M/76.5M [00:01<00:00, 38.9MB/s]
100% 76.5M/76.5M [00:01<00:00, 56.2MB/s]


# 解压数据


In [30]:
!unzip /content/safe_driver_data/porto-seguro-safe-driver-prediction.zip

unzip:  cannot find or open /content/safe_driver_data/porto-seguro-safe-driver-prediction.zip, /content/safe_driver_data/porto-seguro-safe-driver-prediction.zip.zip or /content/safe_driver_data/porto-seguro-safe-driver-prediction.zip.ZIP.


In [7]:
!dir

safe_driver_data  sample_submission.csv.zip  train.csv.zip
sample_data	  test.csv.zip


In [8]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


# GBDT+LR

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomTreesEmbedding,RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,log_loss
from sklearn.pipeline import make_pipeline

In [10]:
df = pd.read_csv('train.csv')
print(df.info())
print('*'*100)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
id                595212 non-null int64
target            595212 non-null int64
ps_ind_01         595212 non-null int64
ps_ind_02_cat     595212 non-null int64
ps_ind_03         595212 non-null int64
ps_ind_04_cat     595212 non-null int64
ps_ind_05_cat     595212 non-null int64
ps_ind_06_bin     595212 non-null int64
ps_ind_07_bin     595212 non-null int64
ps_ind_08_bin     595212 non-null int64
ps_ind_09_bin     595212 non-null int64
ps_ind_10_bin     595212 non-null int64
ps_ind_11_bin     595212 non-null int64
ps_ind_12_bin     595212 non-null int64
ps_ind_13_bin     595212 non-null int64
ps_ind_14         595212 non-null int64
ps_ind_15         595212 non-null int64
ps_ind_16_bin     595212 non-null int64
ps_ind_17_bin     595212 non-null int64
ps_ind_18_bin     595212 non-null int64
ps_reg_01         595212 non-null float64
ps_reg_02         595212 non-null float64
ps_re

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [11]:
df = df.set_index('id')
X = df.iloc[:,1:]
Y = df['target']
print(X.head())
print(Y.head())

    ps_ind_01  ps_ind_02_cat  ...  ps_calc_19_bin  ps_calc_20_bin
id                            ...                                
7           2              2  ...               0               1
9           1              1  ...               1               0
13          5              4  ...               1               0
16          0              1  ...               0               0
17          0              2  ...               1               0

[5 rows x 57 columns]
id
7     0
9     0
13    0
16    0
17    0
Name: target, dtype: int64


In [12]:
#数据清洗
#采用Z-score规范化
ss = StandardScaler(with_mean=False)
X_train = ss.fit_transform(X)
X_train

array([[1.00817252, 3.00935811, 1.85192065, ..., 0.        , 0.        ,
        2.77550942],
       [0.50408626, 1.50467905, 2.59268891, ..., 0.        , 2.09792519,
        0.        ],
       [2.52043131, 6.01871621, 3.33345717, ..., 0.        , 2.09792519,
        0.        ],
       ...,
       [0.50408626, 1.50467905, 3.7038413 , ..., 0.        , 0.        ,
        0.        ],
       [2.52043131, 3.00935811, 1.11115239, ..., 2.21020315, 0.        ,
        0.        ],
       [0.        , 1.50467905, 2.96307304, ..., 0.        , 0.        ,
        0.        ]])

In [0]:
#拆分训练集，测试集
x_train,x_test,y_train,y_test = train_test_split(X_train,Y,test_size=0.5)
#训练集在分为两个部分（GBDT/RF,LR）
x_train,x_train_lr,y_train,y_train_lr = train_test_split(x_train,y_train,test_size=0.5)

In [14]:
#GBDT监督变换
n_estimator = 100
grd = GradientBoostingClassifier(n_estimators=n_estimator)
grd.fit(x_train,y_train)
#OneHot编码
grd_enc = OneHotEncoder(categories='auto')
np.set_printoptions(threshold=np.inf)
grd_enc.fit(grd.apply(x_train)[:,:,0])
#使用OneHot编码作为特征，训练LR
grd_lm = LogisticRegression(solver='lbfgs',max_iter=1000)
grd_lm.fit(grd_enc.transform(grd.apply(x_train_lr)[:,:,0]),y_train_lr)
#使用lr预测
y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(x_test)[:,:,0]))
y_pred_grd_lm[:,1][0:10]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0.02277644, 0.03738731, 0.05021328, 0.09023246, 0.03681978,
       0.04251896, 0.02121776, 0.02107197, 0.01933884, 0.0384819 ])

In [16]:
print(log_loss(y_test,y_pred_grd_lm[:,1],normalize=True))

0.1535601576100027
